In [0]:
#SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("UseCase").getOrCreate()


In [0]:
def get_latest_data(table_name,timestamp_col):
    latest_df = spark.sql(f"SELECT * FROM delta.`{table_name}` WHERE {timestamp_col} = (SELECT MAX({timestamp_col}) FROM delta.`{table_name}`)")
    return latest_df
    

In [0]:
def dq_checks(df):
    df1=df.dropDuplicates()
    df2=df1.dropna(subset="name")
    return df2


In [0]:
table_name="/delta/ingest_table"
timestamp_col="timestamp"

try:
    df=get_latest_data(table_name,timestamp_col)
    dq_df=dq_checks(df)
    #dq_df.printSchema()

    ##Write data into dq_table
    dq_df.write.partitionBy("timestamp")\
                .mode("append")\
                .format("delta")\
                .option("overwriteSchema", "true")\
                .save("/delta/dq_table")
except Exception as e:
    raise Exception(e)
